In [1]:
# -*- coding: utf-8 -*-
# MultiPerceptron
# queueを使った学習

import os
import time
import pandas as pd
import tensorflow as tf
import tensorflow.contrib.slim as slim
import threading
from sklearn.utils import shuffle

tf.reset_default_graph()

MODEL_DIR = "../model_car_lidar_queue"
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)

n_nodes_hl1 = 500
n_nodes_hl2 = 500
n_nodes_hl3 = 500

data_cols = 3 # センサーの数。left45,front,right45
n_classes = 4 # 予測結果の数。stop,left,forward,right
batch_size = 100 # バッチサイズは10〜100前後に
chunk_size = 100 # FIFOQueueのcapacity

TRAIN_NUM=100000 # 学習データの行数
TRAIN_START_INDEX=0
TRAIN_END_INDEX=batch_size
TRAIN_INDEX_I=0
# 学習データを読み込み pandasで読み込みnumpy.ndarrayに変換する
csv_file="../train_data/car_sensor_train_data.csv"
train_dataset = pd.io.parsers.read_csv(csv_file, header = 0, float_precision = "high").values
def next_train_data(batch_size):
    global TRAIN_START_INDEX
    global TRAIN_END_INDEX
    global TRAIN_NUM
    global TRAIN_INDEX_I
    global train_dataset

    if len(train_dataset) < TRAIN_NUM:
        TRAIN_NUM = len(train_dataset)
        
    TRAIN_START_INDEX=TRAIN_INDEX_I*batch_size
    TRAIN_END_INDEX=TRAIN_START_INDEX + batch_size
    if TRAIN_NUM < TRAIN_START_INDEX:
        TRAIN_START_INDEX=0
        TRAIN_END_INDEX=TRAIN_START_INDEX + batch_size
        TRAIN_INDEX_I=0
    if TRAIN_NUM < TRAIN_END_INDEX:
        TRAIN_END_INDEX=TRAIN_NUM

    TRAIN_INDEX_I+=1

    if TRAIN_START_INDEX==0:
        # 先頭に戻った時は学習データをシャッフルする
        train_dataset = shuffle(train_dataset, random_state=42) # shuffle train data
    
    batch_data = train_dataset[TRAIN_START_INDEX:TRAIN_END_INDEX,0:data_cols]
    batch_target = train_dataset[TRAIN_START_INDEX:TRAIN_END_INDEX,data_cols:]
    return batch_data, batch_target


TEST_NUM=820000 # テストデータの行数
TEST_START_INDEX=0
TEST_END_INDEX=batch_size
TEST_INDEX_I=0
# csv読み込み pandasで読み込みnumpy.ndarrayに変換する
csv_file="../train_data/car_sensor_test_data.csv"
test_dataset = pd.io.parsers.read_csv(csv_file, header = 0, float_precision = "high").values
def next_test_data(batch_size):
    global TEST_START_INDEX
    global TEST_END_INDEX
    global TEST_NUM
    global TEST_INDEX_I
    global test_dataset

    if len(test_dataset) < TEST_NUM:
        TEST_NUM = len(test_dataset)
        
    TEST_START_INDEX=TEST_INDEX_I*batch_size
    TEST_END_INDEX=TEST_START_INDEX + batch_size
    if TEST_NUM < TEST_START_INDEX:
        TEST_START_INDEX=0
        TEST_END_INDEX=TEST_START_INDEX + batch_size
        TEST_INDEX_I=0
    if TEST_NUM < TEST_END_INDEX:
        TEST_END_INDEX=TEST_NUM

    TEST_INDEX_I+=1

    # テストデータは精度検証用なのでシャッフルは不要
    batch_data = test_dataset[TEST_START_INDEX:TEST_END_INDEX,0:data_cols]
    batch_target = test_dataset[TEST_START_INDEX:TEST_END_INDEX,data_cols:]
    return batch_data, batch_target


def load_and_enqueue(sess):
    while True:
        try:
            batch_data, batch_target = next_train_data(batch_size)
            sess.run(enqueue_op, feed_dict={placeholder_input_data:batch_data, placeholder_input_target:batch_target})
        except tf.errors.CancelledError as e:
            break
    print("finished enqueueing")
            
with tf.variable_scope("input"):
    placeholder_input_data = tf.placeholder('float', [None, data_cols], name='input_data') # for load_and_enqueue. use dequeue_data_op for prediction
    placeholder_input_target = tf.placeholder('float', name='input_target') # for load_and_enqueue. use dequeue_target_op for prediction
    placeholder_batch_size = tf.placeholder(tf.int32, name='batch_size') # need feed_dict in training sess.run(). don't need for prediction.

with tf.variable_scope("queue"):
    queue = tf.FIFOQueue(
        capacity=chunk_size, # enqueue size
        dtypes=['float', 'float'],
        #dtypes=['float'],
        shapes=[[data_cols], [n_classes]],
        name='FIFOQueue'
    )

    # Enqueue and dequeue operations
    enqueue_op = queue.enqueue_many([placeholder_input_data, placeholder_input_target], name='enqueue_op')
    dequeue_data_op, dequeue_target_op = queue.dequeue_many(placeholder_batch_size, name='dequeue_op') # instead of data/target placeholder


with tf.variable_scope('neural_network_model'):
    hidden_1_layer = {'weights':tf.Variable(tf.random_normal([data_cols, n_nodes_hl1])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))}

    hidden_2_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl2]))}

    hidden_3_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl3]))}

    output_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                    'biases':tf.Variable(tf.random_normal([n_classes])),}


    l1 = tf.add(tf.matmul(dequeue_data_op,hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.relu(l1)

    l2 = tf.add(tf.matmul(l1,hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2)

    l3 = tf.add(tf.matmul(l2,hidden_3_layer['weights']), hidden_3_layer['biases'])
    l3 = tf.nn.relu(l3)

    # 予測結果
    prediction = tf.add(tf.matmul(l3,output_layer['weights']), output_layer['biases'], name='output_y')
    
with tf.variable_scope('loss'):
    losses = tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=dequeue_target_op)
    loss_op = tf.reduce_mean(losses, name='cost')

with tf.variable_scope('accuracy'):
    correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(dequeue_target_op, 1))
    accuracy = tf.reduce_mean(tf.cast(correct, 'float'), name='accuracy')
    accuracy = tf.Print(accuracy, data=[accuracy], message="accuracy:")


train_op = tf.train.AdamOptimizer(0.0001).minimize(loss_op, name='train_op')
max_step = 10
num_batchs = int(TRAIN_NUM/batch_size)

saver = tf.train.Saver()
test_data, test_target =next_test_data(820000)
with tf.Session() as sess:
    ckpt = tf.train.get_checkpoint_state(MODEL_DIR)
    if ckpt:
        # checkpointファイルから最後に保存したモデルへのパスを取得する
        last_model = ckpt.model_checkpoint_path
        print(("load {0}".format(last_model)))
        # 学習済みモデルを読み込む
        saver.restore(sess, last_model)
        LOAD_MODEL = True
    else:
        print("initialization")
        # 初期化処理
        init_op = tf.global_variables_initializer()
        sess.run(init_op)

    start_time, start_clock = time.time(), time.clock()

    # Start a thread to enqueue data asynchronously, and hide I/O latency.
    coord = tf.train.Coordinator()
    enqueue_thread = threading.Thread(target=load_and_enqueue, args=[sess])
    enqueue_thread.isDaemon()
    enqueue_thread.start()
    threads = tf.train.start_queue_runners(coord=coord, sess=sess)
    try:
        # check the accuracy before training (without feed_dict!)
        print sess.run(accuracy, feed_dict={placeholder_batch_size:chunk_size}) # check batch_size's data
        for step in range(max_step):
            batch_loss=0
            for batch_step in range(num_batchs): # 分割したミニバッチを全て実行する
                _, loss = sess.run([train_op, loss_op],
                                   feed_dict={placeholder_batch_size:batch_size})
                batch_loss += loss
            ac = sess.run(accuracy, feed_dict={placeholder_batch_size:chunk_size}) # check batch_size's data

            # 全テストデータでの精度を確認する
            full_test_accuracy = accuracy.eval({'queue/dequeue_op:0':test_data,
                                                'queue/dequeue_op:1':test_target})
            print("Step:%d batch_step:%d accuracy:%.8f full_accuracy:%.8f loss:%.8f time:%.8f clock:%.14f" % (step,batch_step,ac,full_test_accuracy,batch_loss,time.time()-start_time,time.clock()-start_clock))

        sess.run(queue.close(cancel_pending_enqueues=True))
    except Exception, e:
        # Report exceptions to the coodinator.
        print(e)
        coord.request_stop(e)
    finally:
        coord.request_stop()
        coord.join(threads)

    saver.save(sess, MODEL_DIR + '/model.ckpt')


    # 全学習データでの精度を確認する
    test_data, test_target =next_train_data(TRAIN_NUM)
    print('Accuracy:',accuracy.eval({dequeue_data_op:test_data,
                                     dequeue_target_op:test_target}))



print("end")


initialization
0.35
Step:0 batch_step:999 accuracy:0.74000001 full_accuracy:0.67982495 loss:23594208.15509033 time:3.98414302 clock:5.72735800000000
Step:1 batch_step:999 accuracy:0.78999996 full_accuracy:0.80477321 loss:1337108.27764893 time:7.46573400 clock:11.06167300000000
Step:2 batch_step:999 accuracy:0.75000000 full_accuracy:0.78525400 loss:894285.46324158 time:10.88447309 clock:16.34360600000000
Step:3 batch_step:999 accuracy:0.89000005 full_accuracy:0.87581837 loss:752136.58118439 time:14.27854800 clock:21.60757200000000
Step:4 batch_step:999 accuracy:0.85999990 full_accuracy:0.86199784 loss:644476.08745575 time:17.57454610 clock:26.81658400000000
Step:5 batch_step:999 accuracy:0.83999997 full_accuracy:0.89235246 loss:575422.39131165 time:21.05216789 clock:32.11714500000000
Step:6 batch_step:999 accuracy:0.90000010 full_accuracy:0.91491079 loss:540871.18269730 time:24.52354789 clock:37.40650500000000
Step:7 batch_step:999 accuracy:0.81999999 full_accuracy:0.86265850 loss:52286

Step:63 batch_step:999 accuracy:0.93000007 full_accuracy:0.95157796 loss:380268.21091354 time:219.59512210 clock:339.78336300000001
Step:64 batch_step:999 accuracy:0.99000001 full_accuracy:0.96184230 loss:381041.54459643 time:223.05983806 clock:345.10366300000004
Step:65 batch_step:999 accuracy:0.95000005 full_accuracy:0.95019525 loss:379896.82683372 time:226.45467210 clock:350.38233000000002
Step:66 batch_step:999 accuracy:0.95999998 full_accuracy:0.94813627 loss:357106.36274272 time:229.79718709 clock:355.61042100000003
Step:67 batch_step:999 accuracy:0.94000006 full_accuracy:0.95185095 loss:377443.79484783 time:233.23239589 clock:360.93803900000000
Step:68 batch_step:999 accuracy:0.95000005 full_accuracy:0.95235318 loss:400075.96032871 time:236.63367605 clock:366.22852500000005
Step:69 batch_step:999 accuracy:0.93000001 full_accuracy:0.94624579 loss:361896.66134953 time:239.99416590 clock:371.49223400000005
Step:70 batch_step:999 accuracy:0.96000004 full_accuracy:0.93857414 loss:412

Step:125 batch_step:999 accuracy:0.98000002 full_accuracy:0.96929359 loss:284580.98260986 time:430.21022701 clock:665.71496400000001
Step:126 batch_step:999 accuracy:0.97000003 full_accuracy:0.96022409 loss:361218.98572657 time:433.65564895 clock:670.98379799999998
Step:127 batch_step:999 accuracy:1.00000000 full_accuracy:0.97099608 loss:329480.25059727 time:437.06920409 clock:676.24714699999993
Step:128 batch_step:999 accuracy:0.95999998 full_accuracy:0.97302777 loss:285611.42416855 time:440.37987804 clock:681.42706399999997
Step:129 batch_step:999 accuracy:0.95000005 full_accuracy:0.95401204 loss:376497.57714697 time:443.77066994 clock:686.67042400000003
Step:130 batch_step:999 accuracy:0.98999995 full_accuracy:0.96109980 loss:327021.43771821 time:447.09294200 clock:691.84956999999997
Step:131 batch_step:999 accuracy:0.96000004 full_accuracy:0.96961451 loss:314940.51998240 time:450.41672301 clock:697.03683400000000
Step:132 batch_step:999 accuracy:0.97000003 full_accuracy:0.96656775 

Step:187 batch_step:999 accuracy:0.98000002 full_accuracy:0.97330326 loss:317263.50384104 time:645.00715709 clock:993.69255699999997
Step:188 batch_step:999 accuracy:0.99000001 full_accuracy:0.97273982 loss:264742.19454803 time:648.51005292 clock:999.00335999999993
Step:189 batch_step:999 accuracy:0.98999995 full_accuracy:0.96485823 loss:288747.47980912 time:651.99098897 clock:1004.30557099999999
Step:190 batch_step:999 accuracy:0.99000001 full_accuracy:0.97248477 loss:293929.83836851 time:655.54142904 clock:1009.66544799999997
Step:191 batch_step:999 accuracy:0.95000005 full_accuracy:0.96680778 loss:264023.57439989 time:659.06291199 clock:1014.99125600000002
Step:192 batch_step:999 accuracy:0.98000002 full_accuracy:0.96805960 loss:293235.57220977 time:662.63476801 clock:1020.38920200000007
Step:193 batch_step:999 accuracy:1.00000000 full_accuracy:0.97388154 loss:295069.72720274 time:666.19994497 clock:1025.79018399999995
Step:194 batch_step:999 accuracy:0.93000007 full_accuracy:0.9651

Step:249 batch_step:999 accuracy:0.99000001 full_accuracy:0.97539335 loss:242402.12803175 time:867.12159395 clock:1327.06649300000004
Step:250 batch_step:999 accuracy:0.96000004 full_accuracy:0.97784221 loss:257834.24473888 time:870.71949410 clock:1332.46144099999992
Step:251 batch_step:999 accuracy:0.97000003 full_accuracy:0.97204000 loss:242444.79365608 time:874.29532599 clock:1337.83453099999997
Step:252 batch_step:999 accuracy:0.99000001 full_accuracy:0.97663504 loss:224306.58844360 time:877.80868292 clock:1343.16701499999999
Step:253 batch_step:999 accuracy:0.97000003 full_accuracy:0.97053605 loss:253327.60918084 time:881.33939695 clock:1348.49285600000007
Step:254 batch_step:999 accuracy:1.00000000 full_accuracy:0.98131168 loss:264718.47815078 time:884.85298395 clock:1353.82799699999987
Step:255 batch_step:999 accuracy:0.93999994 full_accuracy:0.96468019 loss:220938.92533605 time:888.39956594 clock:1359.18453699999986
Step:256 batch_step:999 accuracy:1.00000000 full_accuracy:0.97

...

Step:19745 batch_step:999 accuracy:1.00000000 full_accuracy:0.99751031 loss:141.06396701 time:67819.26362205 clock:104050.50682299998880
Step:19746 batch_step:999 accuracy:1.00000000 full_accuracy:0.99741143 loss:93.45997384 time:67822.72611809 clock:104055.78288400000019
Step:19747 batch_step:999 accuracy:1.00000000 full_accuracy:0.99743348 loss:63.07884427 time:67826.06777906 clock:104060.98802799999248
Step:19748 batch_step:999 accuracy:1.00000000 full_accuracy:0.99749547 loss:322.57297570 time:67829.37315011 clock:104066.15751799999271
Step:19749 batch_step:999 accuracy:1.00000000 full_accuracy:0.99634343 loss:79.93296227 time:67832.77375889 clock:104071.42177999998967
Step:19750 batch_step:999 accuracy:1.00000000 full_accuracy:0.99416739 loss:121.91821357 time:67836.09775090 clock:104076.62147399999958
Step:19751 batch_step:999 accuracy:1.00000000 full_accuracy:0.99741143 loss:177.09888007 time:67839.55935597 clock:104081.90858699999808
Step:19752 batch_step:999 accuracy:1.0000000

Step:19805 batch_step:999 accuracy:0.99000001 full_accuracy:0.99718827 loss:107.68348669 time:68024.21322107 clock:104365.82523699999729
Step:19806 batch_step:999 accuracy:1.00000000 full_accuracy:0.99710166 loss:156.57833327 time:68027.73037601 clock:104371.16052299999865
Step:19807 batch_step:999 accuracy:1.00000000 full_accuracy:0.99712849 loss:171.91446208 time:68031.15923905 clock:104376.44177999999374
Step:19808 batch_step:999 accuracy:1.00000000 full_accuracy:0.99681389 loss:124.00031206 time:68034.60027790 clock:104381.68986399999994
Step:19809 batch_step:999 accuracy:1.00000000 full_accuracy:0.99660295 loss:227.91344921 time:68038.01134396 clock:104386.93972699998994
Step:19810 batch_step:999 accuracy:1.00000000 full_accuracy:0.99559963 loss:170.56226843 time:68041.43429303 clock:104392.18758099999104
Step:19811 batch_step:999 accuracy:1.00000000 full_accuracy:0.99427652 loss:111.68585653 time:68044.78159595 clock:104397.39572899999621
Step:19812 batch_step:999 accuracy:1.0000

Step:19865 batch_step:999 accuracy:1.00000000 full_accuracy:0.99697721 loss:72.84570678 time:68229.45024109 clock:104681.65430799999740
Step:19866 batch_step:999 accuracy:0.99000001 full_accuracy:0.99660432 loss:118.63591467 time:68232.95307398 clock:104686.95655299999635
Step:19867 batch_step:999 accuracy:0.99000001 full_accuracy:0.99102139 loss:178.82208548 time:68236.39212608 clock:104692.26078899999266
Step:19868 batch_step:999 accuracy:1.00000000 full_accuracy:0.99786156 loss:106.18628149 time:68239.84721303 clock:104697.53233599998930
Step:19869 batch_step:999 accuracy:1.00000000 full_accuracy:0.99664330 loss:119.61643660 time:68243.27596807 clock:104702.80028899999161
Step:19870 batch_step:999 accuracy:1.00000000 full_accuracy:0.99708217 loss:115.56313018 time:68246.74232197 clock:104708.08393199999409
Step:19871 batch_step:999 accuracy:1.00000000 full_accuracy:0.99671274 loss:267.22942411 time:68250.22630095 clock:104713.37117399999988
Step:19872 batch_step:999 accuracy:1.00000

Step:19925 batch_step:999 accuracy:1.00000000 full_accuracy:0.99694359 loss:102.07809551 time:68435.00401497 clock:104997.42424100000062
Step:19926 batch_step:999 accuracy:1.00000000 full_accuracy:0.99706179 loss:213.43775214 time:68438.42362595 clock:105002.68677799998841
Step:19927 batch_step:999 accuracy:1.00000000 full_accuracy:0.99721301 loss:136.80729412 time:68441.86530399 clock:105007.97704399999930
Step:19928 batch_step:999 accuracy:1.00000000 full_accuracy:0.99668354 loss:194.14096398 time:68445.36962700 clock:105013.31249799999932
Step:19929 batch_step:999 accuracy:1.00000000 full_accuracy:0.99702507 loss:152.48602867 time:68448.88512802 clock:105018.64942399998836
Step:19930 batch_step:999 accuracy:1.00000000 full_accuracy:0.99697632 loss:95.34935671 time:68452.39334702 clock:105023.97835499999928
Step:19931 batch_step:999 accuracy:1.00000000 full_accuracy:0.99757373 loss:250.13680983 time:68455.85472512 clock:105029.26007799999206
Step:19932 batch_step:999 accuracy:0.99000

Step:19985 batch_step:999 accuracy:0.98000002 full_accuracy:0.99638152 loss:127.82392108 time:68640.75661302 clock:105313.71246399999654
Step:19986 batch_step:999 accuracy:1.00000000 full_accuracy:0.99761277 loss:211.67938993 time:68644.22573900 clock:105318.99037600000156
Step:19987 batch_step:999 accuracy:1.00000000 full_accuracy:0.99671543 loss:123.61054198 time:68647.71760893 clock:105324.32461199999670
Step:19988 batch_step:999 accuracy:0.99000001 full_accuracy:0.99574584 loss:123.63737431 time:68651.20331502 clock:105329.61806199999410
Step:19989 batch_step:999 accuracy:1.00000000 full_accuracy:0.99792880 loss:98.20005262 time:68654.56808996 clock:105334.84616999998980
Step:19990 batch_step:999 accuracy:1.00000000 full_accuracy:0.99781889 loss:162.28615113 time:68657.99088311 clock:105340.10779399999592
Step:19991 batch_step:999 accuracy:1.00000000 full_accuracy:0.99770057 loss:99.95371746 time:68661.50575209 clock:105345.43343299999833
Step:19992 batch_step:999 accuracy:1.000000